In [1]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

os.chdir('/sharedfolder/sida_classifier/')

speaker_list = ['Male', "Barack Obama", "Female"]

speaker_0_label, speaker_1_label, speaker_2_label = speaker_list

In [3]:
def load_features(dir_path):
    features = []
    for filename in [item for item in os.listdir(dir_path) if 'Weekly' in item]:
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

speaker_1_features = load_features('/sharedfolder/sida_classifier/_classes_Obama_training_clips_all/Barack_Obama/_vowel_mfccs_and_deltas')
print(len(speaker_1_features))

9090


In [4]:
## Load saved features

def load_features(dir_path):
    features = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

#speaker_1_features = load_features('/sharedfolder/sida_classifier/_classes_Obama_training_clips_all/Barack_Obama/_vowel_mfccs_and_deltas')
#print(len(speaker_1_features))

aapb_ubm_male_features = load_features('/sharedfolder/sida_classifier/AAPB_male_vowel_mfccs_and_deltas')
print(len(aapb_ubm_male_features))

aapb_ubm_female_features = load_features('/sharedfolder/sida_classifier/AAPB_female_vowel_mfccs_and_deltas')
print(len(aapb_ubm_female_features))

11531
4382


In [5]:
## Printing MFCCs and deltas for a single frame

print(random.choice(speaker_1_features))

[199.1246130821445, -15.916197831214971, 8.06888339325894, -6.707776169026307, 13.55958556201695, 30.940054584732525, -30.336506576983293, -13.22318064705875, -12.059814945536187, -6.653121348670549, 0.990934623561916, 2.645884251876554, 13.400863776261659, -0.10093933638899166, 2.408304868628142, 2.765767620052355, -1.9764646452984413, -3.5250083410042654, -3.891680776225485, 0.2791892543030234, -4.148409383637479, -2.0274319980205884, -0.5318455712933335, 3.112886297383201, 1.4442263955476373, 1.4032155566282305, 0.29919592974865367, 0.7240451099536601, 0.5839878101901096, -0.0901227036950644, -0.7796283032714333, -0.39944064301346544, -0.07183557826486502, -0.8393208453249937, -0.46262435814681335, 0.04016870704796835, 0.5495166823765689, 0.3170434797209613]


In [83]:
## Combining feature sets

#speaker_1_features = random.sample(speaker_1_features, 10000)

speaker_0_features = aapb_ubm_male_features

speaker_2_features = aapb_ubm_female_features

#print(len(speaker_1_features))
#print(len(ubm_features))

In [86]:
## Training and multi-layer perceptron model with 9/10 of training data and evaluating performance on remaining 1/10

os.chdir('/sharedfolder/sida_classifier/')

import random
random.shuffle(speaker_0_features)
random.shuffle(speaker_1_features)
random.shuffle(speaker_2_features)

from sklearn.neural_network import MLPClassifier

X = speaker_0_features[:-len(speaker_0_features)/10] + speaker_1_features[:-len(speaker_1_features)/10] + speaker_2_features[:-len(speaker_2_features)/10]
y = [1]*len(speaker_0_features[:-len(speaker_0_features)/10]) + [0]*len(speaker_1_features[:-len(speaker_1_features)/10]) + [2]*len(speaker_2_features[:-len(speaker_2_features)/10])

X_train = np.array(X)
y_train = np.array(y)

X_test = speaker_0_features[-len(speaker_0_features)/10:] + speaker_1_features[-len(speaker_1_features)/10:] + speaker_2_features[-len(speaker_2_features)/10:]
y_test = [1]*len(speaker_0_features[-len(speaker_0_features)/10:]) + [0]*len(speaker_1_features[-len(speaker_1_features)/10:]) + [2]*len(speaker_2_features[-len(speaker_2_features)/10:])

X_test = np.array(X_test)
y_test = np.array(y_test)
    
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifier = MLPClassifier().fit(X_train_scaled, y_train)

#classifier = MLPClassifier(max_iter = 2000, random_state = 9, \
#                          hidden_layer_sizes = (100, 100), solver = 'adam', \
#                          activation = 'relu').fit(X_train_scaled, y_train)

print(classifier.score(X_test_scaled,y_test))

0.916666666667


In [10]:
## Training and saving an MLP model with all training data
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

X = speaker_0_features + speaker_1_features + speaker_2_features
y = [0]*len(speaker_0_features) + [1]*len(speaker_1_features) + [2]*len(speaker_2_features)

scaler = StandardScaler()

scaler.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

X_scaled = scaler.transform(X)

#classifier = MLPClassifier().fit(X_scaled, y)

classifier = MLPClassifier(max_iter = 2000, random_state = 9, \
                          hidden_layer_sizes = (100, 100), solver = 'adam', \
                          activation = 'relu').fit(X_scaled, y)

trained_model_filename = 'Barck_Obama' + '_vowels_mlpc_4096_100-5K_w_genders_scaled.pkl'
print(trained_model_filename)

## Saving trained model
joblib.dump(classifier, trained_model_filename)
classifier = joblib.load(trained_model_filename)

Barck_Obama_vowels_mlpc_4096_100-5K_w_genders_scaled.pkl


In [ ]:
##############################################
#### Start here to load pre-trained model ####
##############################################

#trained_model_filename = speaker_1_label.replace(' ', '_') + '_vowels_mlpc_2048.pkl'

#os.chdir('/sharedfolder/sida_classifier/')
#classifier = joblib.load(trained_model_filename)

In [ ]:
%%capture
## Download unseen audio and split into 3-second WAV clips for testing

os.chdir('/sharedfolder/sida_classifier/')

try: os.mkdir('test_clips/')
except: pass

os.chdir('/sharedfolder/sida_classifier/test_clips/')

#!wget -N http://traffic.libsyn.com/wtfpod/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.mp3
#!ffmpeg -i "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.mp3" "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav"
#!ffmpeg -i "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav" -f segment -segment_time 3 "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA_3_sec_%04d.wav"

In [21]:
import scipy.stats

def most_common_class(class_ids):
    mode_id = int(list(scipy.stats.mode(class_ids))[0][0])
    mode_id = 1  #####################################################################################
    mode_id_percentage = float(float(class_ids.count(mode_id))/len(class_ids))
    return (mode_id, mode_id_percentage)

In [58]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

import scipy

os.chdir('/sharedfolder/sida_classifier/test_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = np.array(attk.get_mfccs_and_deltas(wav_pathname))
test_features = scaler.transform(test_features)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation
results_proba = classifier.predict_proba(test_features)  ## Predicting new observation

print(results)
print([round(max(item), 4) for item in list(results_proba)])

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

for i in range(len(results)):
    try:
        if vowel_bools[i]==True:
            vowel_results.append(results[i])
    except: pass
            
display(Audio(wav_pathname))

print("MODE: " + str(list(scipy.stats.mode(results))[0][0])) 
print("MODE vowels only: " + str(list(scipy.stats.mode(vowel_results))[0][0])) ## Vowels only
#print("All samples: "+str(np.mean(results)))
#print("Vowels only: "+str(np.mean(vowel_results)))

mode_id, mode_id_percentage = most_common_class(vowel_results)
top_label = speaker_list[mode_id]

print('')
print("Speaker: " + str(top_label))
print("Confidence: " + str(mode_id_percentage))

print('')

print(str(mode_id) +','+ str(mode_id_percentage) + ',' + str(top_label) + '\n')

/sharedfolder/sida_classifier/test_clips/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA_3_sec_0782.wav
[0 1 0 1 1 0 0 0 0 0 0 1 0 0 2 0 0 1 0 1 1 1 0 0 1 1 2 2 1 0 0 0 0]
[1.0, 0.9998, 1.0, 0.9981, 0.9994, 0.9802, 0.9999, 1.0, 0.998, 0.9994, 0.9976, 0.7027, 0.9955, 0.9995, 0.9203, 1.0, 0.9902, 0.9985, 0.9175, 1.0, 0.9491, 0.7653, 0.9992, 0.8848, 1.0, 0.9102, 0.9995, 0.9377, 0.9978, 1.0, 0.9953, 1.0, 0.8528]


MODE: 0
MODE vowels only: 0

Speaker: Barack Obama
Confidence: 0.291666666667

1,0.291666666667,Barack Obama



In [ ]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

os.chdir('/sharedfolder/sida_classifier/test_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = attk.get_mfccs_and_deltas(wav_pathname)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation

print(results)

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

for i in range(len(results)):
    try:
        if vowel_bools[i]==True:
            vowel_results.append(results[i])
    except:
        pass

display(Audio(wav_pathname))

print("All samples: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

In [59]:
## Function that classifies vowel segments only and returns 
## average output for the full clip

## Function that classifies vowel segments only and returns 
## average output for the full clip

def classify_clip(clip_pathname):
    mfccs = np.array(attk.get_mfccs_and_deltas(clip_pathname))
    mfccs = scaler.transform(mfccs)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = attk.get_vowel_segments(clip_pathname)
    #print(len(mfccs))
    #print(len(results))
    #print(len(vowel_bools))
    
    if len(vowel_bools)==0:
            return most_common_class(results)
        
    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return  most_common_class(vowel_results)[1]   ###### Just the percentage


In [ ]:
%%capture
## Classifying a long audio file

## Writing classification output to CSV

classifier_threshold = 0.55      # Classifier values below this threshold will be discarded


resolution_secs = 3.0

media_path = "/sharedfolder/sida_classifier/test_clips/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav"



os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_Obama_mlpc_4096_100-5K_w_genders_scaled_'+str(resolution_secs)+'s.csv'

counter=0


import timeit
tic=timeit.default_timer()

snd = AudioFileClip.AudioFileClip(media_path)

classifications = []

for i in range(int(attk.duration(media_path)/resolution_secs)):
    try:
        snd.subclip(i * resolution_secs , (i * resolution_secs) + resolution_secs).write_audiofile('/tmp/temp_clip.wav')
        value = classify_clip('/tmp/temp_clip.wav')
        
        with open(csv_path,'a') as fo:
            duration = resolution_secs

            if value >= classifier_threshold:
                start = i * resolution_secs
                fo.write(str(start) + ',' + str(duration) +','+ str(value) + ',' + speaker_1_label + '\n')
        
    except:
        classifications.append(0.0)
        print("Error: " + str(i))

In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
## Writing classification output to CSV

classifier_threshold = 0.2      # Classifier values below this threshold will be discarded

os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_mlpc2048_labels_'+str(resolution_secs)+'s.csv'

counter=0

with open(csv_path,'w') as fo:
    duration = resolution_secs
    for value in classifications:
        if value >= classifier_threshold:
            start = counter * resolution_secs
            fo.write(str(start) + ',' + str(duration) +','+ str(value) + ',' + speaker_1_label + '\n')
        counter+=1